# Chapter 26: Async Patterns

This notebook covers advanced async patterns: async iterators, async generators, async context managers, and the producer-consumer pattern. These patterns enable elegant, composable asynchronous code.

## Key Concepts
- **Async iterators**: Objects with `__aiter__` and `__anext__` for async `for` loops
- **Async generators**: Functions using `async def` with `yield` for lazy async sequences
- **Async context managers**: Objects with `__aenter__` and `__aexit__` for `async with`
- **Producer-consumer**: A common concurrency pattern using `asyncio.Queue`

## Section 1: Async Iterators

An async iterator implements `__aiter__()` and `__anext__()`. It is consumed with `async for` and raises `StopAsyncIteration` when exhausted. This is the async equivalent of the regular iteration protocol.

In [ ]:
import asyncio


class AsyncRange:
    """An async iterator that yields numbers from 0 to stop-1."""

    def __init__(self, stop: int) -> None:
        self.stop = stop
        self.current = 0

    def __aiter__(self):
        return self

    async def __anext__(self) -> int:
        if self.current >= self.stop:
            raise StopAsyncIteration
        value = self.current
        self.current += 1
        return value


async def main() -> None:
    # Use async for to iterate
    print("Async for loop:")
    async for i in AsyncRange(5):
        print(f"  {i}")


asyncio.run(main())

In [ ]:
# Async comprehension with async iterators
async def collect() -> list[int]:
    """Collect async iterator values into a list."""
    return [i async for i in AsyncRange(4)]


result = asyncio.run(collect())
print(f"Collected: {result}")


# Async comprehension with filtering
async def collect_even() -> list[int]:
    return [i async for i in AsyncRange(10) if i % 2 == 0]


evens = asyncio.run(collect_even())
print(f"Even numbers: {evens}")

In [ ]:
# A more realistic async iterator: paginated data fetcher
class AsyncPaginator:
    """Iterate over pages of data asynchronously."""

    def __init__(self, total_items: int, page_size: int) -> None:
        self.total_items = total_items
        self.page_size = page_size
        self.offset = 0

    def __aiter__(self):
        return self

    async def __anext__(self) -> list[int]:
        if self.offset >= self.total_items:
            raise StopAsyncIteration
        # Simulate fetching a page from a database
        await asyncio.sleep(0)
        end = min(self.offset + self.page_size, self.total_items)
        page = list(range(self.offset, end))
        self.offset = end
        return page


async def main() -> None:
    print("Paginated results (page size 3):")
    async for page in AsyncPaginator(total_items=10, page_size=3):
        print(f"  Page: {page}")


asyncio.run(main())

## Section 2: Async Generators

Async generators are the easiest way to create async iterators. They use `async def` with `yield` and can `await` inside the function body. Each `yield` suspends the generator until the next value is requested.

In [ ]:
# Simple async generator
async def async_range(stop: int):
    """Yield numbers 0..stop-1 with an async pause."""
    for i in range(stop):
        await asyncio.sleep(0)  # Simulate async work
        yield i


async def main() -> None:
    # Use async for to consume the generator
    values: list[int] = []
    async for val in async_range(5):
        values.append(val)
    print(f"Values: {values}")

    # Or use an async comprehension
    collected = [i async for i in async_range(3)]
    print(f"Collected: {collected}")


asyncio.run(main())

In [ ]:
# Async generator with filtering and transformation
from typing import AsyncGenerator


async def fetch_items(count: int) -> AsyncGenerator[dict[str, int | str], None]:
    """Simulate fetching items from an async data source."""
    for i in range(count):
        await asyncio.sleep(0)
        yield {"id": i, "name": f"item-{i}", "value": i * 10}


async def filter_valuable(
    items: AsyncGenerator[dict[str, int | str], None],
    min_value: int,
) -> AsyncGenerator[dict[str, int | str], None]:
    """Filter items by minimum value."""
    async for item in items:
        if item["value"] >= min_value:
            yield item


async def main() -> None:
    print("Items with value >= 30:")
    async for item in filter_valuable(fetch_items(8), min_value=30):
        print(f"  {item}")


asyncio.run(main())

In [ ]:
# Async generator pipeline: compose multiple transformations
async def numbers(n: int) -> AsyncGenerator[int, None]:
    """Generate numbers 0..n-1."""
    for i in range(n):
        await asyncio.sleep(0)
        yield i


async def squared(source: AsyncGenerator[int, None]) -> AsyncGenerator[int, None]:
    """Square each value."""
    async for val in source:
        yield val ** 2


async def only_even(source: AsyncGenerator[int, None]) -> AsyncGenerator[int, None]:
    """Keep only even values."""
    async for val in source:
        if val % 2 == 0:
            yield val


async def main() -> None:
    # Build a pipeline: numbers -> squared -> only_even
    pipeline = only_even(squared(numbers(8)))
    results = [val async for val in pipeline]
    print(f"Even squares of 0..7: {results}")


asyncio.run(main())

## Section 3: Async Context Managers

Async context managers implement `__aenter__` and `__aexit__` and are used with `async with`. They manage resources that require async setup and teardown, like database connections or file handles.

In [ ]:
# Class-based async context manager
class AsyncResource:
    """A resource with async setup and teardown."""

    async def __aenter__(self):
        print("  Entering: acquiring resource...")
        await asyncio.sleep(0)  # Simulate async setup
        return self

    async def __aexit__(self, *args: object) -> None:
        print("  Exiting: releasing resource...")
        await asyncio.sleep(0)  # Simulate async teardown

    async def do_work(self) -> str:
        return "work done"


async def main() -> None:
    async with AsyncResource() as resource:
        result = await resource.do_work()
        print(f"  Inside: {result}")
    print("  After: resource released")


asyncio.run(main())

In [ ]:
# Tracking lifecycle events
events: list[str] = []


class TrackedResource:
    """An async context manager that records lifecycle events."""

    async def __aenter__(self):
        events.append("enter")
        return self

    async def __aexit__(self, *args: object) -> None:
        events.append("exit")


async def main() -> None:
    async with TrackedResource():
        events.append("use")


asyncio.run(main())
print(f"Events: {events}")
assert events == ["enter", "use", "exit"]

In [ ]:
# Function-based async context manager using contextlib
from contextlib import asynccontextmanager
from typing import AsyncGenerator


@asynccontextmanager
async def async_timer(label: str) -> AsyncGenerator[None, None]:
    """Time an async block of code."""
    import time
    start = time.perf_counter()
    print(f"  [{label}] Starting...")
    try:
        yield
    finally:
        elapsed = time.perf_counter() - start
        print(f"  [{label}] Done in {elapsed:.4f}s")


async def main() -> None:
    async with async_timer("fetch"):
        await asyncio.sleep(0.05)

    async with async_timer("compute"):
        await asyncio.sleep(0.02)


asyncio.run(main())

In [ ]:
# Practical example: async database connection mock
@asynccontextmanager
async def async_db_connection(
    dsn: str,
) -> AsyncGenerator["MockConnection", None]:
    """Simulate an async database connection."""
    conn = MockConnection(dsn)
    await conn.connect()
    try:
        yield conn
    finally:
        await conn.close()


class MockConnection:
    """A mock async database connection."""

    def __init__(self, dsn: str) -> None:
        self.dsn = dsn
        self.connected = False

    async def connect(self) -> None:
        await asyncio.sleep(0)
        self.connected = True
        print(f"  Connected to {self.dsn}")

    async def close(self) -> None:
        await asyncio.sleep(0)
        self.connected = False
        print(f"  Disconnected from {self.dsn}")

    async def query(self, sql: str) -> list[dict[str, str]]:
        await asyncio.sleep(0)
        return [{"result": f"data from: {sql}"}]


async def main() -> None:
    async with async_db_connection("postgres://localhost/mydb") as conn:
        rows = await conn.query("SELECT * FROM users")
        print(f"  Query result: {rows}")
    # Connection is automatically closed here


asyncio.run(main())

## Section 4: Producer-Consumer Pattern

The producer-consumer pattern uses `asyncio.Queue` to decouple producers (which generate work) from consumers (which process it). This is a foundational pattern for building async pipelines.

In [ ]:
# Basic producer-consumer with asyncio.Queue
async def producer(
    queue: asyncio.Queue[int],
    count: int,
) -> None:
    """Produce items and put them on the queue."""
    for i in range(count):
        await asyncio.sleep(0.01)  # Simulate production time
        await queue.put(i)
        print(f"  Produced: {i}")


async def consumer(
    queue: asyncio.Queue[int],
    name: str,
    results: list[int],
) -> None:
    """Consume items from the queue."""
    while True:
        item = await queue.get()
        await asyncio.sleep(0.02)  # Simulate processing time
        results.append(item * 10)
        print(f"  {name} processed: {item} -> {item * 10}")
        queue.task_done()


async def main() -> None:
    queue: asyncio.Queue[int] = asyncio.Queue(maxsize=5)
    results: list[int] = []

    # Start 2 consumers
    consumers = [
        asyncio.create_task(consumer(queue, f"Worker-{i}", results))
        for i in range(2)
    ]

    # Run producer
    await producer(queue, count=6)

    # Wait for all items to be processed
    await queue.join()

    # Cancel consumers (they loop forever)
    for c in consumers:
        c.cancel()

    print(f"\nAll results: {sorted(results)}")


asyncio.run(main())

In [ ]:
# Producer-consumer with sentinel value for graceful shutdown
SENTINEL: None = None


async def data_producer(
    queue: asyncio.Queue[int | None],
    items: list[int],
    num_consumers: int,
) -> None:
    """Produce data, then send sentinel to each consumer."""
    for item in items:
        await asyncio.sleep(0)
        await queue.put(item)

    # Signal each consumer to stop
    for _ in range(num_consumers):
        await queue.put(SENTINEL)


async def data_consumer(
    queue: asyncio.Queue[int | None],
    name: str,
) -> list[int]:
    """Consume until sentinel is received."""
    processed: list[int] = []
    while True:
        item = await queue.get()
        if item is SENTINEL:
            print(f"  {name}: received shutdown signal")
            break
        processed.append(item ** 2)
        await asyncio.sleep(0)
    return processed


async def main() -> None:
    queue: asyncio.Queue[int | None] = asyncio.Queue()
    num_consumers = 2

    # Start producer and consumers
    producer_task = asyncio.create_task(
        data_producer(queue, [1, 2, 3, 4, 5, 6], num_consumers)
    )
    consumer_tasks = [
        asyncio.create_task(data_consumer(queue, f"Consumer-{i}"))
        for i in range(num_consumers)
    ]

    # Wait for all to finish
    await producer_task
    all_results = await asyncio.gather(*consumer_tasks)

    # Combine results from all consumers
    combined = sorted(r for results in all_results for r in results)
    print(f"\nAll squared values: {combined}")


asyncio.run(main())

## Section 5: Combining Patterns

Real-world async code combines these patterns. Here we build a mini pipeline that uses async generators, context managers, and queues together.

In [ ]:
# Combining async generator + context manager + queue
from typing import AsyncGenerator


async def event_stream(count: int) -> AsyncGenerator[dict[str, int | str], None]:
    """Simulate an async event stream."""
    for i in range(count):
        await asyncio.sleep(0)
        yield {"id": i, "type": "click" if i % 2 == 0 else "scroll"}


@asynccontextmanager
async def event_processor(
    name: str,
) -> AsyncGenerator[list[dict[str, int | str]], None]:
    """Context manager for collecting processed events."""
    processed: list[dict[str, int | str]] = []
    print(f"  [{name}] Processor started")
    try:
        yield processed
    finally:
        print(f"  [{name}] Processor done: {len(processed)} events")


async def main() -> None:
    async with event_processor("ClickFilter") as results:
        async for event in event_stream(8):
            if event["type"] == "click":
                results.append(event)

    print(f"  Click events: {results}")


asyncio.run(main())

In [ ]:
# Async for with set/dict comprehensions
async def user_ids(count: int) -> AsyncGenerator[int, None]:
    """Generate user IDs with some duplicates."""
    for i in range(count):
        await asyncio.sleep(0)
        yield i % 5  # Creates duplicates


async def main() -> None:
    # Async set comprehension — deduplicates automatically
    unique_ids: set[int] = {uid async for uid in user_ids(10)}
    print(f"Unique IDs: {sorted(unique_ids)}")

    # Async dict comprehension
    user_map: dict[int, str] = {
        uid: f"user-{uid}" async for uid in user_ids(5)
    }
    print(f"User map: {user_map}")


asyncio.run(main())

## Section 6: Error Handling in Async Patterns

Errors in async iterators and context managers follow the same patterns as their synchronous counterparts, but the `async` versions require special handling for cleanup.

In [ ]:
# Async context manager handles exceptions in __aexit__
class SafeResource:
    """A resource that always cleans up, even on error."""

    def __init__(self, name: str) -> None:
        self.name = name

    async def __aenter__(self):
        print(f"  {self.name}: acquired")
        return self

    async def __aexit__(
        self,
        exc_type: type[BaseException] | None,
        exc_val: BaseException | None,
        exc_tb: object,
    ) -> bool:
        if exc_val:
            print(f"  {self.name}: error occurred - {exc_val}")
        print(f"  {self.name}: released")
        return False  # Don't suppress the exception


async def main() -> None:
    # Normal usage
    print("Normal case:")
    async with SafeResource("DB") as db:
        print(f"  Using {db.name}")

    # Error case — resource still cleaned up
    print("\nError case:")
    try:
        async with SafeResource("DB") as db:
            raise RuntimeError("connection lost")
    except RuntimeError:
        print("  Handled the error")


asyncio.run(main())

In [ ]:
# Async generator cleanup with try/finally
async def resilient_stream(
    items: list[str],
) -> AsyncGenerator[str, None]:
    """An async generator with guaranteed cleanup."""
    print("  Stream: opened")
    try:
        for item in items:
            await asyncio.sleep(0)
            yield item
    finally:
        print("  Stream: closed (cleanup done)")


async def main() -> None:
    # Normal: iterate to completion
    print("Full iteration:")
    async for val in resilient_stream(["a", "b", "c"]):
        print(f"    got: {val}")

    # Early exit: break triggers cleanup
    print("\nEarly exit (break):")
    async for val in resilient_stream(["x", "y", "z"]):
        print(f"    got: {val}")
        if val == "x":
            break


asyncio.run(main())

## Summary

### Async Iterators
- Implement `__aiter__()` and `__anext__()`
- Raise `StopAsyncIteration` when exhausted
- Consumed with `async for` or async comprehensions

### Async Generators
- Easiest way to create async iterators: `async def` + `yield`
- Can `await` inside the generator body
- Composable: chain generators into pipelines
- Support cleanup via `try`/`finally`

### Async Context Managers
- Implement `__aenter__()` and `__aexit__()` for class-based
- Use `@asynccontextmanager` decorator for function-based
- Used with `async with` for resource management
- `__aexit__` is called even when exceptions occur

### Producer-Consumer
- `asyncio.Queue` decouples producers from consumers
- `queue.put()` / `queue.get()` are async operations
- Use `queue.join()` + `task_done()` for completion tracking
- Sentinel values enable graceful shutdown

### Key Patterns
- Use async generators for lazy, memory-efficient async sequences
- Use `@asynccontextmanager` for simple resource management
- Combine patterns for real-world async pipelines
- Always ensure cleanup with `try`/`finally` or context managers